In [1]:
from findmycells.main import Project

In [2]:
project_root_dir = '/mnt/c/Users/dsege/TEMP/test_project/'

In [3]:
test_project = Project(project_root_dir)

In [6]:
test_project.database.deepflash2_dir

'/mnt/c/Users/dsege/TEMP/test_project/03_deepflash2/'